## CSKG Setup, Installation and Downloads

In [ ]:
import os
# Clone repos
os.chdir('/home/jaleed/Jaleed/SSG-VQA/SSG')
if os.path.isdir('cskg')==False:
    !git clone https://github.com/usc-isi-i2/cskg
os.chdir('cskg')
if os.path.isdir('grounding/graphify')==False:
    !git clone https://github.com/ucinlp/mowgli-uci
    !mv mowgli-uci/* grounding/
# Remove versions from packge names in dependencies to minimize conflict
if os.path.isfile('requirements1.txt')==False:
    file2 = open("requirements1.txt","w")
    with open("requirements.txt", "r") as file1:
        for pkgver in file1:
            if 'kgtk' in pkgver:
                continue
            if 'demjson' in pkgver:
                continue
            if '==' in pkgver:
                [pkg, ver] = pkgver.split('==')
                file2.write(pkg+'\n')
            if ' @ ' in pkgver:
                [pkg, ver] = pkgver.split(' @ ')
                file2.write(pkg+'\n')
    file1.close()
    file2.close()

if os.path.isfile('grounding/requirements1.txt')==False:
    file2 = open("grounding/requirements1.txt","w")
    with open("grounding/requirements.txt", "r") as file1:
        for pkgver in file1:
            if '==' in pkgver:
                [pkg, ver] = pkgver.split('==')
                file2.write(pkg+'\n')
    file1.close()
    file2.close()

In [ ]:
!pip install --upgrade pip setuptools==57.5.0
!pip install --upgrade python-pushover
!python -m pip install kgtk==0.5.0
!python -m pip install -r requirements1.txt --no-cache-dir
!python -m pip install -r grounding/requirements1.txt --no-cache-dir
!conda install --yes faiss-cpu -c pytorch #-n mowgli
!python -m spacy download en_core_web_lg
!python -m pip install kgtk==0.5.0
!python -m pip install altair

In [ ]:
!kgtk --version
!kgtk query -h

In [ ]:
# Downloads:
# 1) download all files from https://drive.google.com/drive/u/1/folders/16347KHSloJJZIbgC9V5gH7_pRx0CzjPQ
# and place in 'cskg/output' folder, and unzip all
# 2) move BERT embeddings ('bert_nli_large_w2v_format.txt.gz') to 'cskg/output/embeddings'
# 3) download numberbatch
# !wget https://conceptnet.s3.amazonaws.com/downloads/2019/numberbatch/numberbatch-19.08.txt.gz -P output/embeddings/

# Unzip Downloads:
#!gunzip -k output/*.txt.gz
#!gunzip -k output/*.tsv.gz
#!gunzip -k output/embeddings/*.txt.gz

# Run these in terminal:
# sudo apt-get update -y
# sudo apt-get upgrade -y
# sudo apt-get install -y graphviz
# python -m pip install graphviz
# sudo apt-get install -y xdg-utils

In [ ]:
%cd /home/jaleed/Jaleed/SSG-VQA/SSG/cskg

/home/jaleed/Jaleed/SSG-VQA/SSG/cskg


In [ ]:
# Parameters
cskg_path = "output" #"../output"
kg = "cskg_connected.kgtk.gz" #kg = "cskg.tsv.gz"
nkg = "cskg-normalized.kgtk.gz"
delete_database = "yes"

In [ ]:
import io
import os
import subprocess
import sys
import numpy as np
import pandas as pd
import altair as alt

In [ ]:
os.environ['CSKG'] = cskg_path
os.environ['KG'] = "{}/{}".format(cskg_path, kg)
os.environ['NKG'] = "{}/{}".format(cskg_path, nkg)
os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cskg_path)
os.environ['kypher'] = "kgtk query --graph-cache " + os.environ['STORE']

In [ ]:
def bar_chart(data, x_column, y_column, title="", width=800):
    """Construct a simple bar chart with two properties"""
    bars = alt.Chart(data).mark_bar().encode(
        y=alt.Y(y_column, sort='-x'),
        x=x_column
    ).properties(
        title=title,
        width=width
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=x_column
    )

    return (bars + text)

import io
import pandas
import subprocess

def shell_df(command, shell=False, **kwargs):
    """
    Takes a shell command as a string and and reads the result into a Pandas DataFrame.

    Additional keyword arguments are passed through to pandas.read_csv.

    :param command: a shell command that returns tabular data
    :type command: str
    :param shell: passed to subprocess.Popen
    :type shell: bool

    :return: a pandas dataframe
    :rtype: :class:`pandas.dataframe`
    """
    proc = subprocess.Popen(command,
                            shell=shell,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)
    output, error = proc.communicate()

    if proc.returncode == 0:
        if error:
            print(error.decode())
        with io.StringIO(output.decode()) as buffer:
            return pandas.read_csv(buffer, **kwargs)
    else:
        message = ("Shell command returned non-zero exit status: {0}\n\n"
                   "Command was:\n{1}\n\n"
                   "Standard error was:\n{2}")
        raise IOError(message.format(proc.returncode, command, error.decode()))

In [ ]:
# CSKG Embeddings
graph_emb = "trans_log_dot_0.1.tsv.gz" #graph embedding output file - TransE
text_emb = "cskg_embeddings_bert_nli_large.txt.gz" #text embedding output file - BERT
distance='cosine' #embedding distance metric

In [ ]:
import os
from pathlib import Path
import gzip
import pickle as pkl
import faiss
import numpy as np
from typing import Callable, List, Tuple
import json
import hashlib
import logging
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
os.environ['CSKG'] = cskg_path
os.environ['GE'] = "{}/{}".format(cskg_path, graph_emb)
os.environ['TE'] = "{}/{}".format(cskg_path, text_emb)
graph_emb_path = os.environ['GE']
text_emb_path = os.environ['TE']
!echo $CSKG
!echo $GE
!echo $TE

output
output/trans_log_dot_0.1.tsv.gz
output/cskg_embeddings_bert_nli_large.txt.gz


In [ ]:
# Utilities

class Vocab:
    def __init__(self, words) -> None:
        self.idx_to_word = words
        self.word_to_idx = {word: idx for idx, word in enumerate(words)}

def read_embedding_file(embedding_file: Path, dim: int, emb_col=1) -> Tuple[Vocab, np.ndarray]:

    logger.debug(f'Reading embeddings from {embedding_file}')

    shape = tuple([count_lines(embedding_file), dim])

    with gzip.open(embedding_file, 'r') as f:

        embeddings = np.zeros(shape, dtype=np.float32)

        if emb_col!=1:
            header=next(f)
        i=0
        words = []
        for line in tqdm(f, total=shape[0]):
            line=line.decode()
            if emb_col==1:
                node, *embedding = line.split()
            else:
                line_data=line.split()
                if line_data[1]=='embedding_sentence': continue
                node=line_data[0]
                embedding=line_data[2].split(',')
            embedding = np.array([float(x) for x in embedding])
            words.append(node)
            embeddings[i] = embedding
            i+=1

    vocab = Vocab(words)

    return vocab, embeddings

def count_lines(embedding_file: Path):
    with gzip.open(embedding_file, 'r') as f:
        i=0
        for line in f:
            i+=1
    return i

def build_index(metric: str, embeddings: np.ndarray):

    logger.debug(f'Building search index')

    if metric == 'cosine':
        index = faiss.IndexFlatIP(embeddings.shape[-1])
    elif metric == 'l2':
        index = faiss.IndexFlatL2(embeddings.shape[-1])
    else:
        raise ValueError(f'Bad metric: {metric}')

    index.add(embeddings)

    return index

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
# Load graph embeddings
graph_dim = 100 # Dimension of the graph embeddings for our example's file
graph_vocab, graph_embeddings = read_embedding_file(graph_emb_path,graph_dim)
graph_index = build_index(distance, graph_embeddings)

100%|██████████████████████████████| 2160968/2160968 [01:29<00:00, 24086.51it/s]


In [ ]:
# Load text embeddings
#text_dim = 1024 # Dimension of the text embeddings for our example's file
#text_vocab, text_embeddings = read_embedding_file(text_emb_path, text_dim, emb_col=2)
#text_index = build_index(distance, text_embeddings)

In [ ]:
import time
for i in range(0,1):
    t = time.time()
    command = "$kypher -i $NKG --limit 10 \
                --match '(n1)-[r]->(n2)' \
                --where 'n1 = \"" + '/c/en/person' + "\" AND n2=\"" + '/c/en/racket' + "\"'"
#   --where 'n1 = \"" + '/c/en/woman' + "\" AND r.label=\"" + '/r/RelatedTo' + "\"'"
    stats = shell_df(command, shell=True, sep='\t')
    print(str(time.time()-t)+' seconds.') # 25sec
    print(stats)

In [ ]:
command = "$kypher -i $NKG --limit 25 \
                --match '(n1)-[r]->(n2)' \
                --where 'n1 = \"" + '/c/en/door' + "\" AND n2=\"" + '/c/en/house' + "\"'"
stats = shell_df(command, shell=True, sep='\t')
print(stats)

In [ ]:
command = "$kypher -i $NKG \
                --match '(n1)-[r]->(n2)' \
                --where 'n1 = \"" + '/c/en/helmet' + "\"'"
stats = shell_df(command, shell=True, sep='\t')
print(stats)

## Scene Graph Generation

In [ ]:
import os
import copy
import sys
import torch
import h5py
import json
import shutil
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import random
from maskrcnn_benchmark.structures.bounding_box import BoxList
from maskrcnn_benchmark.structures.boxlist_ops import boxlist_iou
eval_inp_img = '/home/jaleed/Jaleed/SSG-VQA/Eval_IO/gqa/0_images'
eval_outp_sg = '/home/jaleed/Jaleed/SSG-VQA/Eval_IO/gqa/1_pred_scene_graphs'
%cd /home/jaleed/Jaleed/SSG-VQA/SSG/Scene

for img_file in os.listdir(eval_inp_img):
    if os.path.isdir(eval_outp_sg+'/'+img_file)==False:
        print(img_file)
        # clear SGG input folder
        !rm -r /home/jaleed/Jaleed/SSG-VQA/SSG/temp_dir_inp
        os.mkdir('/home/jaleed/Jaleed/SSG-VQA/SSG/temp_dir_inp')
        # copy image to SGG input directory from eval_inp_img directory
        shutil.copyfile(eval_inp_img+'/'+img_file, '/home/jaleed/Jaleed/SSG-VQA/SSG/temp_dir_inp/'+img_file)
        # run SGG
        !(CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port 10027 --nproc_per_node=1 tools/relation_test_net.py \
            --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
            MODEL.ROI_RELATION_HEAD.USE_GT_BOX False \
            MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
            MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
            MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE none \
            MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum \
            MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \
            TEST.IMS_PER_BATCH 1 DTYPE "float16" \
            GLOVE_DIR /home/jaleed/Jaleed/SSG-VQA/SSG/glove \
            MODEL.PRETRAINED_DETECTOR_CKPT /home/jaleed/Jaleed/SSG-VQA/SSG/checkpoint/upload_causal_motif_sgdet \
            OUTPUT_DIR /home/jaleed/Jaleed/SSG-VQA/SSG/checkpoint/upload_causal_motif_sgdet \
            TEST.CUSTUM_EVAL True \
            TEST.CUSTUM_PATH /home/jaleed/Jaleed/SSG-VQA/SSG/temp_dir_inp \
            DETECTED_SGG_DIR /home/jaleed/Jaleed/SSG-VQA/SSG/temp_dir_out)
        # copy result from SGG output directory to eval_outp_sg directory
        shutil.copytree('/home/jaleed/Jaleed/SSG-VQA/SSG/temp_dir_out', eval_outp_sg+'/'+img_file)

        # reset (to avoid mem/IO error) and repeat imports and variables after reset
        %reset -f
        import os
        import copy
        import sys
        import torch
        import h5py
        import json
        import shutil
        from matplotlib.pyplot import imshow
        from PIL import Image, ImageDraw
        import numpy as np
        from collections import defaultdict
        from tqdm import tqdm
        import random
        from maskrcnn_benchmark.structures.bounding_box import BoxList
        from maskrcnn_benchmark.structures.boxlist_ops import boxlist_iou
        eval_inp_img = '/home/jaleed/Jaleed/SSG-VQA/Eval_IO/gqa/0_images' #repeat for gqa, flickr and coco
        eval_outp_sg = '/home/jaleed/Jaleed/SSG-VQA/Eval_IO/gqa/1_pred_scene_graphs'
        %cd /home/jaleed/Jaleed/SSG-VQA/SSG/Scene

/home/jaleed/Jaleed/SSG-VQA/SSG/Scene
2408978.jpg
2022-08-10 17:51:08,713 maskrcnn_benchmark INFO: Using 1 GPUs
2022-08-10 17:51:08,713 maskrcnn_benchmark INFO: AMP_VERBOSE: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 4
  SIZE_DIVISIBILITY: 32
DATASETS:
  TEST: ('GQA_stanford_filtered_with_attribute_test',)
  TO_TEST: None
  TRAIN: ('GQA_stanford_filtered_with_attribute_train',)
  VAL: ('GQA_stanford_filtered_with_attribute_val',)
DETECTED_SGG_DIR: /home/jaleed/Jaleed/SSG-VQA/SSG/temp_dir_out
DTYPE: float16
GLOVE_DIR: /home/jaleed/Jaleed/SSG-VQA/SSG/glove
INPUT:
  BRIGHTNESS: 0.0
  CONTRAST: 0.0
  HUE: 0.0
  MAX_SIZE_TEST: 1000
  MAX_SIZE_TRAIN: 1000
  MIN_SIZE_TEST: 600
  MIN_SIZE_TRAIN: (600,)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  SATURATION: 0.0
  TO_BGR255: True
  VERTICAL_FLIP_PROB_TRAIN: 0.0
MODEL:
  ATTRIBUTE_ON: False
  BACKBONE:
    CONV_BODY: R-101-FPN
    FREEZE_CONV_BODY_AT: 2
  CLS_AGNOSTIC_BBOX_REG: False
  DEVICE: 

In [ ]:
#dir_eval_io = '/home/jaleed/Jaleed/SSG-VQA/Eval_IO/vg/'
dir_eval_io = '/home/jaleed/Jaleed/SSG-VQA/Eval_IO/gqa/'

In [ ]:
def draw_single_box(pic, box, color='red', draw_info=None):
    draw = ImageDraw.Draw(pic)
    x1,y1,x2,y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
    draw.rectangle(((x1, y1), (x2, y2)), outline=color)
    if draw_info:
        draw.rectangle(((x1, y1), (x1+50, y1+10)), fill=color)
        info = draw_info
        draw.text((x1, y1), info)

def print_list(name, input_list, scores=None):
    for i, item in enumerate(input_list):
        if scores == None:
            print(name + ' ' + str(i) + ': ' + str(item))
        else:
            print(name + ' ' + str(i) + ': ' + str(item) + '; score: ' + str(scores[i]))

def draw_image(img_path, boxes, box_labels, rel_labels, box_scores=None, rel_scores=None):
    size = get_size(Image.open(img_path).size)
    pic = Image.open(img_path).resize(size)
    num_obj = len(boxes)
    for i in range(num_obj):
        info = str(i) + '_' + box_labels[i]
        draw_single_box(pic, boxes[i], draw_info=info)
    display(pic)
    print('*' * 50)
    print_list('box_labels', box_labels, box_scores)
    print('*' * 50)
    print_list('rel_labels', rel_labels, rel_scores)

    return None

def get_size(image_size):
    min_size = 600
    max_size = 1000
    w, h = image_size
    size = min_size
    if max_size is not None:
        min_original_size = float(min((w, h)))
        max_original_size = float(max((w, h)))
        if max_original_size / min_original_size * size > max_size:
            size = int(round(max_size * min_original_size / max_original_size))
    if (w <= h and w == size) or (h <= w and h == size):
        return (w, h)
    if w < h:
        ow = size
        oh = int(size * h / w)
    else:
        oh = size
        ow = int(size * w / h)
    return (ow, oh)

In [ ]:
from PIL import Image
import json
def load_sg(img_path, box_topk):

    img = Image.open(img_path)
    custom_prediction_file = open(f'{dir_eval_io}1_pred_scene_graphs/'+img_path.split('0_images/')[1]+'/custom_prediction.json')
    custom_prediction = json.load(custom_prediction_file)
    custom_data_info_file = open(f'{dir_eval_io}1_pred_scene_graphs/'+img_path.split('0_images/')[1]+'/custom_data_info.json')
    custom_data_info = json.load(custom_data_info_file)

    image_idx = 0
    #box_topk = 5 # select top k bounding boxes, now taken as func arg
    rel_topk = int(np.math.factorial(box_topk)/np.math.factorial(box_topk-2)) # select top k relationships
    ind_to_classes = custom_data_info['ind_to_classes']
    ind_to_predicates = custom_data_info['ind_to_predicates']
    boxes = custom_prediction[str(image_idx)]['bbox'][:box_topk]
    box_labels = custom_prediction[str(image_idx)]['bbox_labels'][:box_topk]
    box_scores = custom_prediction[str(image_idx)]['bbox_scores'][:box_topk]
    all_rel_labels = custom_prediction[str(image_idx)]['rel_labels']
    all_rel_scores = custom_prediction[str(image_idx)]['rel_scores']
    all_rel_pairs = custom_prediction[str(image_idx)]['rel_pairs']

    for i in range(len(box_labels)):
        box_labels[i] = ind_to_classes[box_labels[i]]

    rel_labels = []
    rel_scores = []
    for i in range(len(all_rel_pairs)):
        if all_rel_pairs[i][0] < box_topk and all_rel_pairs[i][1] < box_topk:
            rel_scores.append(all_rel_scores[i])
            label = str(all_rel_pairs[i][0]) + '_' + box_labels[all_rel_pairs[i][0]] + ' => ' + ind_to_predicates[all_rel_labels[i]] + ' => ' + str(all_rel_pairs[i][1]) + '_' + box_labels[all_rel_pairs[i][1]]
            rel_labels.append(label)

    rel_labels = rel_labels[:rel_topk]
    rel_scores = rel_scores[:rel_topk]

    #draw_image(img, boxes, box_labels, rel_labels, box_scores=box_scores, rel_scores=rel_scores)

    return box_labels,rel_labels,boxes

In [ ]:
import graphviz
def draw_graph(nodes,edges):

    #init graph
    g = graphviz.Digraph('G',format='png')
    g.splines='true'
    g.overlap='false'

    if len(edges)>0:
        # prepare nodes/edges
        if type(edges[0])==str:
            # if input params are (box_labels,rel_labels) from SGG
            edges1=list()
            for i in range(0,len(edges)):
                edge = {}
                r1,r2,r3 = edges[i].split(' => ')
                [r11,r12] = r1.split('_')
                edge['node1_id'] = r11
                edge['node1'] = r12
                edge['node1_sg'] = True
                edge['rel'] = r2
                edge['rel_sg'] = True
                [r31,r32] = r3.split('_')
                edge['node2_id'] = r31
                edge['node2'] = r32
                edge['node2_sg'] = True
                edges1.append(edge)
        else:
            # if input params are (nodes,edges) from CSKG
            edges1 = edges

        # build the graph
        for i in range(0,len(edges1)):
            e = edges1[i]
            if e['node1_sg'] == True:
                g.node(e['node1_id'],label=e['node1'],shape='rect',style='filled',fillcolor='black',fontcolor='white')
            else:
                g.node(e['node1_id'],label=e['node1'],shape='ellipse')
            if e['node2_sg'] == True:
                g.node(e['node2_id'],label=e['node2'],shape='rect',style='filled',fillcolor='black',fontcolor='white')
            else:
                g.node(e['node2_id'],label=e['node2'],shape='ellipse')
            if e['rel_sg'] == True:
                g.edge(e['node1_id'],e['node2_id'],label=e['rel'])
            else:
                g.edge(e['node1_id'],e['node2_id'],label=e['rel'],dir='none') # (CSKG undirected edge)
    return g

## Parsing SG Concepts to CSKG representation

In [ ]:
def node_similarity(node1,node2):

    try:
        node1_emb=graph_embeddings[graph_vocab.word_to_idx[node1]]
    except:
        try:
            [node1,x] = node1.split('_')
            node1_emb=graph_embeddings[graph_vocab.word_to_idx[node1]]
        except:
            return 0.0

    try:
        node2_emb=graph_embeddings[graph_vocab.word_to_idx[node2]]
    except:
        try:
            [node2,x] = node2.split('_')
            node2_emb=graph_embeddings[graph_vocab.word_to_idx[node2]]
        except:
            return 0.0

    return cosine_similarity([node1_emb],[node2_emb])[0][0]

# def word_similarity(node1,node2):

#     try:
#         node1_emb=text_embeddings[text_vocab.word_to_idx[node1]]
#     except:
#         [node1,x] = node1.split('_')
#         node1_emb=text_embeddings[text_vocab.word_to_idx[node1]]

#     try:
#         node2_emb=text_embeddings[text_vocab.word_to_idx[node2]]
#     except:
#         [node2,x] = node2.split('_')
#         node2_emb=text_embeddings[text_vocab.word_to_idx[node2]]

#     return cosine_similarity([node1_emb],[node2_emb])[0][0]

In [ ]:
def parse_to_cskg(box_labels,rel_labels):
    nodes = list()
    edges = list()
    for i in range(0,len(box_labels)):
        node={}
        node['id'] = str(i)
        node['label'] = '/c/en/' + box_labels[i].replace(' ','_')
        node['sg'] = True
        nodes.append(node)
    for i in range(0,len(rel_labels)):
        edge = {}
        rel_node = {}
        r1,r2,r3 = rel_labels[i].split(' => ')
        [r11,r12] = r1.split('_')
        node1_id = r11
        node1 = '/c/en/'+r12.replace(' ','_')
        rel = '/r/'+r2.replace(' ','_')
        [r31,r32] = r3.split('_')
        node2_id = r31
        node2 = '/c/en/'+r32.replace(' ','_')
        edge['node1']=node1
        edge['node1_id']=node1_id
        edge['node1_sg']=True
        edge['rel']=rel
        edge['rel_sg']=True
        edge['node2']=node2
        edge['node2_id']=node2_id
        edge['node2_sg']=True
        edges.append(edge)
    return nodes,edges

## Graph Enrichment

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [ ]:
def refine_graph(nodes,edges,boxes,box_labels):
    # first identify nodes to remove based on graph embedding and location
    # in case of similarity, the 2nd node is noted for removal from nodes and edges
    # noted edges/nodes are removed
    # finally, edges involving no corresponding sg_rel_id are also removed
    idx = list()
    for i in range(0,len(box_labels)-1):
        obj_node1 = '/c/en/' + box_labels[i].replace(' ','_')
        obj_node1_box = boxes[i]
        for j in range(i+1,len(box_labels)):
            obj_node2 = '/c/en/' + box_labels[j].replace(' ','_')
            obj_node2_box = boxes[j]
            if node_similarity(obj_node1,obj_node2)>0.5 or bb_intersection_over_union(obj_node1_box,obj_node2_box)>0.5:
                idx.append(j)
    idx = list(set(idx)) # sort the list and remove duplicates
    for i in range(0,len(idx)):
        node_id = str(idx[i])
        # remove this node
        j=0
        while j < len(nodes):
            if nodes[j]['id']==node_id:
                nodes.pop(j)
            else:
                j=j+1
        # remove triplets involving this node
        j=0
        while j < len(edges):
            if edges[j]['node1_id']==node_id or edges[j]['node2_id']==node_id:
                edges.pop(j)
            else:
                j=j+1
    # remove nodes not in any triplet
    i=0
    while i < len(nodes):
        flag=0
        for edge in edges:
            if nodes[i]['id'] in [edge['node1_id'],edge['node2_id']]:
                flag=1
                break
        if flag==0:
            nodes.pop(i)
        else:
            i=i+1
    return [nodes,edges]

In [ ]:
import json
gqa_rels_json = json.load(open('../relationships.json'))
gqa_dict = json.load(open('../GQA-SGG-dicts-with-attri.json'))
gqa_unique_objs = list(dict.fromkeys(gqa_dict['object_count'].keys())) # 150 unique rels in GQA
gqa_unique_rels = list(dict.fromkeys(gqa_dict['predicate_count'].keys())) # 50 unique rels in GQA

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import csv

rels_dict = gqa_dict['predicate_count']
rels_dict = {k: v for k, v in sorted(rels_dict.items(), reverse=True, key=lambda item: item[1])}
rels_dict = {k: 100*(v/sum(rels_dict.values())) for k, v in rels_dict.items()}
# fig = plt.figure(figsize=[100,70])
# plt.rcParams["font.size"] = 100
# plt.xticks(fontsize = 50, rotation = 60)
# plt.xlabel('Predicate')
# plt.ylabel('Frequency (%)')
# plt.bar(range(len(rels_dict)), rels_dict.values(), tick_label=list(rels_dict.keys()))
# plt.savefig("/home/jaleed/Jaleed/SSG-VQA/freqplot_rels.png")

#objs_dict = gqa_dict['object_count']
# objs_dict = {k: v for k, v in sorted(objs_dict.items(), reverse=True, key=lambda item: item[1])}
# objs_dict = {k: 100*(v/sum(objs_dict.values())) for k, v in objs_dict.items()}
# fig2 = plt.figure(figsize=[140,60])
# plt.rcParams["font.size"] = 100
# plt.xticks(fontsize = 50, rotation = 90)
# plt.xlabel('Object')
# plt.ylabel('Frequency (%)')
# plt.bar(range(len(objs_dict)), objs_dict.values(), tick_label=list(objs_dict.keys()))
# plt.savefig("/home/jaleed/Jaleed/SSG-VQA/freqplot_objs.png")

l = list()
l.append(rels_dict)
with open('gqa_rels_freq.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = rels_dict.keys())
    writer.writeheader()
    writer.writerows(l)

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment=None
import time
def extract_from_cskg(nodes):
    new_nodes_rels = list()
    for node in nodes:
        t = time.time()

        # Part 1/2: (node, --, --)
        command = "$kypher -i $NKG \
                    --match '(n1)-[]->()' \
                    --where '(n1 = \"" + node['label'] + "\")'"
        stats=shell_df(command, shell=True, sep='\t')
        stats.drop(stats[stats['label']=='label'].index, inplace=True)
        # skip the ones that have same node on both sides (e.g. man-similarTo-man)
        stats.drop(stats[stats['node1']==stats['node2']].index, inplace=True)
        # calculate similarity of nodes in each row
        stats['similarity']=np.zeros((len(stats),1))
        for i in stats.index:
            stats['similarity'][i] = node_similarity(stats['node1'][i],stats['node2'][i])
        # remove duplicate rows
        stats.drop_duplicates()
        # sort stats in descending order by similarity score
        stats.sort_values(by=['similarity'], inplace=True, ascending=False)
        rels = list(pd.unique(stats['label']))
        for rel in rels:
            # one instance of each unique rel
            sub_stats = stats[stats['label']==rel].head(1)
            for i in sub_stats.index:
                if sub_stats.loc[i]['similarity']>=0.5:
                    new_nodes_rels.append([sub_stats.loc[i]['node1'],sub_stats.loc[i]['label'],
                                            sub_stats.loc[i]['node2'],sub_stats.loc[i]['similarity']])

        # Part 2/2: (--, --, node)
        command = "$kypher -i $NKG \
                    --match '()-[]->(n2)' \
                    --where '(n2 = \"" + node['label'] + "\")'"
        stats=shell_df(command, shell=True, sep='\t')
        stats.drop(stats[stats['label']=='label'].index, inplace=True)
        # skip the ones that have same node on both sides (e.g. man-similarTo-man)
        stats.drop(stats[stats['node1']==stats['node2']].index, inplace=True)
        # calculate similarity of nodes in each row
        stats['similarity']=np.zeros((len(stats),1))
        for i in stats.index:
            stats['similarity'][i] = node_similarity(stats['node1'][i],stats['node2'][i])
        # remove duplicate rows
        stats.drop_duplicates()
        # sort stats in descending order by similarity score
        stats.sort_values(by=['similarity'], inplace=True, ascending=False)
        rels = list(pd.unique(stats['label']))
        for rel in rels:
            # one instance of each unique rel;
            sub_stats = stats[stats['label']==rel].head(1)
            for i in sub_stats.index:
                if sub_stats.loc[i]['similarity']>=0.5:
                    new_nodes_rels.append([sub_stats.loc[i]['node1'],sub_stats.loc[i]['label'],
                                            sub_stats.loc[i]['node2'],sub_stats.loc[i]['similarity']])

        print('Extraction from CSKG done for '+node['label']+' in '+ str(int(time.time()-t)) +' sec)')
    return new_nodes_rels

In [ ]:
def enrich_graph(nodes,edges):

    # extract and add relevant nodes and edges from CSKG
    new_nodes_rels = extract_from_cskg(nodes)

    new_nodes=list()
    new_edges=list()
    for new_node_rel in new_nodes_rels:
        new_node1 = new_node_rel[0]
        new_rel = new_node_rel[1]
        new_node2 = new_node_rel[2]

        # check if the 1st new node is already present
        if (new_node1 in [n['label'] for n in nodes+new_nodes]):
            # if yes, link the new edge to the existing node
            for n in nodes+new_nodes:
                if n['label']==new_node1:
                    new_node1_id = n['id']
                    new_node1_sg = n['sg']
        else:
            # otherwise create new node and then link
            new_node1_id = str(int(nodes[-1]['id'])+len(new_nodes)+1)
            new_node1_sg = False
            new_nodes.append({'id':new_node1_id,
                              'label':new_node1,
                              'sg':new_node1_sg})

        # similarly for 2nd new node
        if (new_node2 in [n['label'] for n in nodes+new_nodes]):
            for n in nodes+new_nodes:
                if n['label']==new_node2:
                    new_node2_id = n['id']
                    new_node2_sg = n['sg']
        else:
            new_node2_id = str(int(nodes[-1]['id'])+len(new_nodes)+1)
            new_node2_sg = False
            new_nodes.append({'id':new_node2_id,
                              'label':new_node2,
                              'sg':new_node2_sg})

        new_edge = {'node1': new_node1,
                    'node1_id': new_node1_id,
                    'node1_sg': new_node1_sg,
                    'rel': new_rel,
                    'rel_sg': False,
                    'node2': new_node2,
                    'node2_id': new_node2_id,
                    'node2_sg':new_node2_sg}
        new_edges.append(new_edge)

    nodes1 = nodes + new_nodes
    edges1 = edges + new_edges
    return nodes1,edges1

## Parsing back to SG format for downstream task or evaluation

In [ ]:
def node_to_sg(node):
    if '/c/en/' in node['label']:
        x = node['label'].split('/')
        node_label = x[3]
        obj_sim_scores = np.zeros((len(gqa_unique_objs),1))
        if node['sg'] == True:
            node['label'] = node_label.replace('_',' ')
        else:
            for i in range(0,len(gqa_unique_objs)):
                obj_sim_scores[i]=node_similarity('/c/en/'+node_label,'/c/en/'+gqa_unique_objs[i].replace(' ','_'))
            if max(obj_sim_scores)>0.5:
                max_index = [n==max(obj_sim_scores) for n in obj_sim_scores].index(True)
                node['label'] = gqa_unique_objs[max_index]
            else:
                node = None
    else:
        node = None
    return node

In [ ]:
import pandas as pd
df = pd.read_excel('../CSKGtoSG-rels.xlsx')
def parse_to_sg(nodes, edges, gqa_unique_objs=gqa_unique_objs, gqa_unique_rels=gqa_unique_rels):
    nodes1=list()
    edges1=list()
    for node in nodes:
        node = node_to_sg(node)
        if (node is not None) and (node['label'] not in [n['label'] for n in nodes1]):
            nodes1.append(node)
    for edge in edges:
        node1 = node_to_sg({'id': edge['node1_id'], 'label': edge['node1'], 'sg': edge['node1_sg']})
        node2 = node_to_sg({'id': edge['node2_id'], 'label': edge['node2'], 'sg': edge['node2_sg']})
        rel = None
        # if both nodes exist
        if (node1 is not None) and (node2 is not None):
            # if node1 or node2 already present, link to it
            if node1['label'] in [node['label'] for node in nodes1]:
                for node in nodes1:
                    if node['label']==node1['label']:
                        node1['id'] = node['id']
                        node1['sg'] = node['sg']
            if node2['label'] in [node['label'] for node in nodes1]:
                for node in nodes1:
                    if node['label']==node2['label']:
                        node2['id'] = node['id']
                        node2['sg'] = node['sg']
            # process rel if different nodes on both sides
            if node1['id']!=node2['id']:
                x = edge['rel'].split('/')
                x = x[2].replace('_',' ')
                if x in gqa_unique_rels:
                    rel = x
                else:
                    for i in df.index:
                        if df['cskg_rel'][i].lower() == edge['rel'].lower():
                            rel = df['sg_rel'].iloc[i]
                if (rel is not None) and str(rel)!='nan':
                    edges1.append({
                        'node1': node1['label'],
                        'node1_id': node1['id'],
                        'node1_sg': node1['sg'],
                        'rel': rel,
                        'rel_sg': edge['rel_sg'],
                        'node2': node2['label'],
                        'node2_id': node2['id'],
                        'node2_sg': node2['sg']
                    })
    # discard nodes that are not used in edges1
    nodes1 = [n for n in nodes1 if n['id'] in list(set([e['node1_id'] for e in edges1]+[e['node2_id'] for e in edges1]))]
    return nodes1, edges1

In [ ]:
def save_sg_format(img_path, nodes, edges):
    custom_prediction_file = open(f'{dir_eval_io}1_pred_scene_graphs/'+img_path.split('0_images/')[1]+'/custom_prediction.json')
    custom_prediction = json.load(custom_prediction_file)
    custom_data_info_file = open(f'{dir_eval_io}1_pred_scene_graphs/'+img_path.split('0_images/')[1]+'/custom_data_info.json')
    custom_data_info = json.load(custom_data_info_file)
    ind_to_classes = custom_data_info['ind_to_classes']
    ind_to_predicates = custom_data_info['ind_to_predicates']
    for node in nodes:
        if node['sg']==False:
            bbox = custom_prediction['0']['bbox']
            bbox.append(bbox[1])
            custom_prediction['0']['bbox'] = bbox
            bbox_labels = custom_prediction['0']['bbox_labels']
            bbox_labels.append(ind_to_classes.index(node['label']))
            custom_prediction['0']['bbox_labels'] = bbox_labels
            bbox_scores = custom_prediction['0']['bbox_scores']
            bbox_scores.append(max(bbox_scores))
            custom_prediction['0']['bbox_scores'] = bbox_scores
    for edge in edges:
        if edge['rel_sg']==False:
            bbox_labels = custom_prediction['0']['bbox_labels']
            node1 = bbox_labels.index(ind_to_classes.index(edge['node1']))
            node2 = bbox_labels.index(ind_to_classes.index(edge['node2']))
            rel_pairs = custom_prediction['0']['rel_pairs']
            rel_pairs.insert(0,[node1,node2])
            custom_prediction['0']['rel_pairs'] = rel_pairs
            rel_labels = custom_prediction['0']['rel_labels']
            rel_labels.insert(0,ind_to_predicates.index(edge['rel']))
            custom_prediction['0']['rel_labels'] = rel_labels
            rel_scores = custom_prediction['0']['rel_scores']
            rel_scores.insert(0,max(rel_scores))
            custom_prediction['0']['rel_scores'] = rel_scores
            rel_all_scores = custom_prediction['0']['rel_all_scores']
            rel_all_scores.insert(0,rel_all_scores[1])
            custom_prediction['0']['rel_all_scores'] = rel_all_scores
    os.mkdir(f'{dir_eval_io}2_enriched_scene_graphs/'+img_path.split('0_images/')[1])
    with open(f'{dir_eval_io}2_enriched_scene_graphs/'+img_path.split('0_images/')[1]+'/custom_prediction.json', 'w') as f:
        json.dump(custom_prediction, f)
    with open(f'{dir_eval_io}2_enriched_scene_graphs/'+img_path.split('0_images/')[1]+'/custom_data_info.json', 'w') as f:
        json.dump(custom_data_info, f)

## Putting it all together

In [ ]:
from PIL import Image
import json

In [ ]:
for filename in os.listdir(f'{dir_eval_io}0_images/'):
    img_path = f'{dir_eval_io}0_images/{filename}'

    if os.path.exists(f'{dir_eval_io}graph_figures/'+img_path.split('0_images/')[1]):
        print('***** EXISTS: '+ img_path +' *****')
        continue
    else:

        print('***** PROCESSING '+ img_path +' *****')

        # Load scene graph
        [box_labels,rel_labels,boxes] = load_sg(img_path, 10) # 2nd arg is max objects

        # Draw scene graph
        g = draw_graph(box_labels,rel_labels)
        g.render(f'{dir_eval_io}graph_figures/'+img_path.split('0_images/')[1]+'/01_sg')

        # Parse to CSKG
        [nodes1,edges1] = parse_to_cskg(box_labels,rel_labels)
        g1 = draw_graph(nodes1,edges1)
        g1.render(f'{dir_eval_io}graph_figures/'+img_path.split('0_images/')[1]+'/02_sg_parsed')

        # Refine graph
        [nodes2,edges2]=refine_graph(nodes1,edges1,boxes,box_labels)
        g2 = draw_graph(nodes2,edges2)
        g2.render(f'{dir_eval_io}graph_figures/'+img_path.split('0_images/')[1]+'/03_sg_parsed_refined')

        # Enrich graph
        [nodes3,edges3] = enrich_graph(nodes2,edges2)
        g3 = draw_graph(nodes3,edges3)
        g3.render(f'{dir_eval_io}graph_figures/'+img_path.split('0_images/')[1]+'/04_sg_parsed_refined_enriched')

        # Parse to SG
        [nodes4,edges4] = parse_to_sg(nodes3,edges3)
        g4 = draw_graph(nodes4,edges4)
        g4.render(f'{dir_eval_io}graph_figures/'+img_path.split('0_images/')[1]+'/05_sg_parsed_refined_enriched_reverseparsed')

        # Save final graph as JSON
        graph = {'nodes':nodes4, 'edges':edges4}
        with open(f'{dir_eval_io}graph_figures/'+img_path.split('0_images/')[1]+'.json', 'w') as f:
            json.dump(graph, f)

        # Save the final result in SGG format
        save_sg_format(img_path, nodes4, edges4)
    print('\n')

## Generating Groundtruth

In [ ]:
for filename in os.listdir(f'{dir_eval_io}0_images'):
    img_path = f'{dir_eval_io}/0_images/{filename}'
    img_id = int(img_path.split('0_images/')[1].split('.jpg')[0])
    box_labels = list()
    rel_labels = list()
    if os.path.exists(f'{dir_eval_io}0_gt_scene_graphs/'+img_path.split('0_images/')[1]+'_gt_gqa'):
        #print('***** EXISTS: '+ img_path +' *****')
        continue
    else:
        #print('***** PROCESSING '+ img_path +' *****')
        for img in gqa_rels_json:
            if img['image_id'] == img_id:
                for rel in img['relationships']:
                    if rel['predicate'].lower() in gqa_unique_rels:
                        # in relationships.json, there is either 'name' or 'names' key for subject and object
                        pred = rel['predicate'].lower()
                        if 'name' in rel['subject'].keys():
                            if 'name' in rel['object'].keys():
                                subj = rel['subject']['name']
                                obj = rel['object']['name']
                            else:
                                subj = rel['subject']['name']
                                obj = rel['object']['names'][0]
                        else:
                            if 'name' in rel['object'].keys():
                                subj = rel['subject']['names'][0]
                                obj = rel['object']['name']
                            else:
                                subj = rel['subject']['names'][0]
                                obj = rel['object']['names'][0]
                        if (subj in gqa_unique_objs) and (obj in gqa_unique_objs):
                            if subj in box_labels:
                                subj_id = box_labels.index(subj)
                            else:
                                box_labels.append(subj)
                                subj_id = box_labels.index(subj)
                            if obj in box_labels:
                                obj_id = box_labels.index(obj)
                            else:
                                box_labels.append(obj)
                                obj_id = box_labels.index(obj)
                            rel_labels.append(str(subj_id)+'_'+subj +' => '+ pred +' => '+ str(obj_id)+'_'+obj)
        g = draw_graph(box_labels,rel_labels)
        g.render(f'{dir_eval_io}0_gt_scene_graphs/'+img_path.split('0_images/')[1]+'_gt_gqa')

## Evaluation on GQA

In [ ]:
import json, os, numpy as np, copy
os.chdir('/home/jaleed/Jaleed')
gqa_rels_json = json.load(open('CSKG/relationships.json'))
gqa_dict = json.load(open('CSKG/GQA-SGG-dicts-with-attri.json'))
gqa_unique_objs = list(dict.fromkeys(gqa_dict['object_count'].keys()))
gqa_unique_rels = list(dict.fromkeys(gqa_dict['predicate_count'].keys()))

In [ ]:
results = {}

for dataset in ['gqa']:

    overall_recall = list()
    overall_recall_cs = list()
    overall_m_recall = list()
    overall_m_recall_cs = list()

    for K in [50,100]:

        pred_sg_dir = f'Eval_IO/{dataset}/1_pred_scene_graphs/'
        pred_sg_dir_cs = f'Eval_IO/{dataset}/2_enriched_scene_graphs/'
        assert(os.listdir(pred_sg_dir) == os.listdir(pred_sg_dir_cs))

        recall = list() #R@K for scene graphs
        recall_cs = list() #R@K for enriched scene graphs
        m_recall = list() #mR@K for scene graphs
        m_recall_cs = list() #mR@K for enriched scene graphs

        l = os.listdir(pred_sg_dir)
        #l[-2:] = []
        #l=l[0:5]
        for pred_sg in l: #['2317213.jpg']
            # 1/3 Retrieve groundtruth predicates
            gt_rels = [img['relationships'] for img in gqa_rels_json if img['image_id']==int(pred_sg.split('.')[0])][0]
            gt_predicates = list()
            for gt_rel in gt_rels:
                if 'name' in gt_rel['subject'].keys():
                    obj1 = gt_rel['subject']['name']
                else:
                    obj1 = gt_rel['subject']['names'][0]
                if 'name' in gt_rel['object'].keys():
                    obj2 = gt_rel['object']['name']
                else:
                    obj2 = gt_rel['object']['names'][0]
                rel = gt_rel['predicate'].lower()

                if rel in gqa_unique_rels:
                    gt_predicates.append(rel)

            # Retrieve keys (idnices of obj/pred labels)
            custom_data_info = json.load(open(pred_sg_dir+pred_sg+'/custom_data_info.json'))
            ind_to_classes = custom_data_info['ind_to_classes']
            ind_to_predicates = custom_data_info['ind_to_predicates']

            # 2/3 Retrieve predicted predicates (without commonsense)
            custom_prediction = json.load(open(pred_sg_dir+pred_sg+'/custom_prediction.json'))
            bbox_labels = custom_prediction['0']['bbox_labels']
            bbox_scores = custom_prediction['0']['bbox_scores']
            rel_pairs = custom_prediction['0']['rel_pairs']
            rel_labels = custom_prediction['0']['rel_labels']
            rel_scores = custom_prediction['0']['rel_scores']
            pred_objects = [ind_to_classes[ind] for ind in bbox_labels]
            pred_object_pairs = [[pred_objects[inds[0]],pred_objects[inds[1]]] for inds in rel_pairs]
            pred_predicates = [ind_to_predicates[ind] for ind in rel_labels]
            assert(len(pred_object_pairs)==len(pred_predicates))
            k_pred_predicates = pred_predicates[:min(K,len(pred_predicates))] # Top K Predicted Predicates

            # Compute Recall@K
            count_preds = 0 # number of correct predicted predicates
            gt_predicates1=copy.deepcopy(gt_predicates)
            for i in range(0,len(k_pred_predicates)):
                if k_pred_predicates[i] in gt_predicates1:
                    count_preds += 1
                    gt_predicates1.pop(gt_predicates1.index(k_pred_predicates[i]))

            if len(gt_predicates)>0:
                recall.append(count_preds*100/len(gt_predicates))
            else:
                #pass
                recall.append(0)

            # Computer mR@K
            gqa_unique_rels_recall = list()
            for i in range(0,50):
                k_pred_predicates1 = [p for p in k_pred_predicates if p==gqa_unique_rels[i]] # subset of k_pred_predicates containing only i-th unique_rel
                gt_predicates1 = [p for p in gt_predicates if p==gqa_unique_rels[i]] # subset of gt_predicates containing only i-th unique_rel
                if len(gt_predicates1)==0 or len(k_pred_predicates1)==0:
                    gqa_unique_rels_recall.append(0)
                else:
                    gqa_unique_rels_recall.append(len(k_pred_predicates1)*100/max(len(gt_predicates1),len(k_pred_predicates1)))
            m_recall.append(np.mean(gqa_unique_rels_recall))


            # 3/3 Retrieve predicted predicates (with commonsense)
            custom_prediction = json.load(open(pred_sg_dir_cs+pred_sg+'/custom_prediction.json'))
            bbox_labels = custom_prediction['0']['bbox_labels']
            rel_pairs = custom_prediction['0']['rel_pairs']
            rel_labels = custom_prediction['0']['rel_labels']
            rel_scores = custom_prediction['0']['rel_scores']
            pred_objects = [ind_to_classes[ind] for ind in bbox_labels]
            pred_object_pairs = [[pred_objects[inds[0]],pred_objects[inds[1]]] for inds in rel_pairs]
            pred_predicates = [ind_to_predicates[ind] for ind in rel_labels]
            assert(len(pred_object_pairs)==len(pred_predicates))
            k_pred_predicates = pred_predicates[:min(K,len(pred_predicates))] # Top K Predicted Predicates

            # Compute Recall@K
            count_preds = 0 # number of correct predicted predicates
            gt_predicates2=copy.deepcopy(gt_predicates)
            for i in range(0,len(k_pred_predicates)):
                if k_pred_predicates[i] in gt_predicates2:
                    count_preds += 1
                    gt_predicates2.pop(gt_predicates2.index(k_pred_predicates[i]))

            if len(gt_predicates)>0:
                recall_cs.append(count_preds*100/len(gt_predicates))
            else:
                #pass
                recall_cs.append(0)

            # Computer mR@K
            gqa_unique_rels_recall = list()
            for i in range(0,50):
                k_pred_predicates1 = [p for p in k_pred_predicates if p==gqa_unique_rels[i]] # subset of k_pred_predicates containing only i-th unique_rel
                gt_predicates1 = [p for p in gt_predicates if p==gqa_unique_rels[i]] # subset of gt_predicates containing only i-th unique_rel
                if len(gt_predicates1)==0 or len(k_pred_predicates1)==0:
                    gqa_unique_rels_recall.append(0)
                else:
                    gqa_unique_rels_recall.append(len(k_pred_predicates1)*100/max(len(gt_predicates1),len(k_pred_predicates1)))
            m_recall_cs.append(np.mean(gqa_unique_rels_recall))

        print(K, 'done')

        overall_recall.append(sum(recall)/len([r for r in recall]))
        overall_recall_cs.append(sum(recall_cs)/len([r for r in recall_cs]))
        overall_m_recall.append(sum(m_recall)/len([r for r in m_recall]))
        overall_m_recall_cs.append(sum(m_recall_cs)/len([r for r in m_recall_cs]))

    print(dataset, 'done')
    results[dataset] = [overall_recall, overall_recall_cs, overall_m_recall, overall_m_recall_cs]
json.dump(results, open(f'Eval_IO/{dataset}/results.json','w'))

50 done
100 done
gqa done


In [2]:
results = json.load(open(f'Eval_IO/{dataset}/results1.json'))
print(results)

{'gqa': [[29.2172208937719, 32.69647206780441], [36.031804134886141, 41.721526939683707], [10.190612309145152, 12.11504329083056], [12.380390450797544, 15.104173263905842]]}
